In [33]:
import tensorflow
import stellargraph
import numpy
import matplotlib
import os.path
import pandas

Please notice, to open jupyter in custom directory use:
>jupyter notebook --notebook-dir=[your_directory]

In [34]:
data_path = "../data/"
file_list = [name for name in os.listdir(data_path)]
print(file_list)

['Sesja 1.xlsx', 'Sesja 10.xlsx', 'Sesja 11.xlsx', 'Sesja 12.xlsx', 'Sesja 13.xlsx', 'Sesja 14.xlsx', 'Sesja 15.xlsx', 'Sesja 16.xlsx', 'Sesja 17.xlsx', 'Sesja 18.xlsx', 'Sesja 19.xlsx', 'Sesja 2.xlsx', 'Sesja 20.xlsx', 'Sesja 21.xlsx', 'Sesja 22.xlsx', 'Sesja 23.xlsx', 'Sesja 24.xlsx', 'Sesja 25.xlsx', 'Sesja 26.xlsx', 'Sesja 3.xlsx', 'Sesja 4.xlsx', 'Sesja 5.xlsx', 'Sesja 6.xlsx', 'Sesja 7.xlsx', 'Sesja 8.xlsx', 'Sesja 9.xlsx']


In [39]:
# Sessions order is important, let me get sorted file info
sorted_filenames = dict()
for name in file_list:
    list_of_digits = list(map(int, filter(str.isdigit, name)))
    sorted_filenames[name] = list_of_digits[0] if len(list_of_digits) == 1 else list_of_digits[1] + list_of_digits[0]*10
sorted_filenames = sorted(sorted_filenames.items(), key=lambda el: el[1])
sorted_filenames = {val: key for key, val in sorted_filenames}
print(sorted_filenames)

{1: 'Sesja 1.xlsx', 2: 'Sesja 2.xlsx', 3: 'Sesja 3.xlsx', 4: 'Sesja 4.xlsx', 5: 'Sesja 5.xlsx', 6: 'Sesja 6.xlsx', 7: 'Sesja 7.xlsx', 8: 'Sesja 8.xlsx', 9: 'Sesja 9.xlsx', 10: 'Sesja 10.xlsx', 11: 'Sesja 11.xlsx', 12: 'Sesja 12.xlsx', 13: 'Sesja 13.xlsx', 14: 'Sesja 14.xlsx', 15: 'Sesja 15.xlsx', 16: 'Sesja 16.xlsx', 17: 'Sesja 17.xlsx', 18: 'Sesja 18.xlsx', 19: 'Sesja 19.xlsx', 20: 'Sesja 20.xlsx', 21: 'Sesja 21.xlsx', 22: 'Sesja 22.xlsx', 23: 'Sesja 23.xlsx', 24: 'Sesja 24.xlsx', 25: 'Sesja 25.xlsx', 26: 'Sesja 26.xlsx'}


In [42]:
# Loading data
data_dict = dict()
for key, val in sorted_filenames.items():
    data_dict[key] = pandas.read_excel(data_path + val)

In [44]:
rand_int = int(numpy.random.rand() * 26)
print(data_dict[rand_int])

                Unnamed: 0  Unnamed: 1  Numer osoby  Unnamed: 3  Unnamed: 4  \
0                  Nr O.B.         1.0          2.0         3.0         4.0   
1                        1      2420.0       2518.0      1232.0      2123.0   
2                        2      2215.0       2206.0      2412.0      2011.0   
3                        3      2021.0       2321.0      2220.0      2119.0   
4                        4      2713.0       2016.0      2321.0      2019.0   
5                        5      3422.0       3120.0      2029.0      2618.0   
6                        7      3509.0       3505.0      3307.0      3408.0   
7                        8      3031.0       2309.0       636.0      1031.0   
8                        9      2823.0       2214.0      2715.0      1723.0   
9                       10      2412.0       2816.0      1722.0      2016.0   
10                      11      1812.0        520.0      3433.0      3228.0   
11                      12      3226.0       2834.0 